In [12]:
if "google.colab" in str(get_ipython()):
    # uninstall preinstalled packages from Colab to avoid conflicts
    !pip uninstall -y torch notebook notebook_shim tensorflow tensorflow-datasets prophet torchaudio torchdata torchtext torchvision
    !pip install neuralprophet

from neuralprophet import NeuralProphet
import mlflow
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

#Set up MLFlow via DAGSHub
os.environ['MLFLOW_TRACKING_USERNAME'] = '2Duffman'
os.environ['MLFLOW_TRACKING_PASSWORD'] = 'f6651f529b37bdd5aa99d6a092e7d48359374423'
os.environ['MLFLOW_TRACKING_PROJECTNAME'] = 'Symbolic-Regression'
mlflow.set_tracking_uri(f'https://dagshub.com/' + os.environ['MLFLOW_TRACKING_USERNAME']
                        + '/' + os.environ['MLFLOW_TRACKING_PROJECTNAME'] + '.mlflow')

Now we will read the data and fix all the missing 00:00:00

In [13]:
# Read the data
data = pd.read_csv("distance_8.csv")

# Define the target column
target = "saldo_final_target"

# Function to check if time is present in the string
def check_time(s):
    first_colon = s.find(':')
    if first_colon == -1:
        return s + ' 00:00:00'
    else:
        return s

# Apply the function to the 'delivery_start' column
data['delivery_start'] = data['delivery_start'].apply(check_time)

# Convert 'delivery_start' to datetime
data['delivery_start'] = pd.to_datetime(data['delivery_start'], format='%Y-%m-%d %H:%M:%S')

# Create 'year' column
data['year'] = data['delivery_start'].dt.year

# Create 'day' column with day of the year
data['day'] = data['delivery_start'].dt.dayofyear

# Create 'time' column with minutes since midnight
data['time'] = data['delivery_start'].dt.hour * 60 + data['delivery_start'].dt.minute

# Create cyclic representations of 'day' and 'time'
data['day_sin'] = np.sin(2 * np.pi * data['day'] / 365)
data['day_cos'] = np.cos(2 * np.pi * data['day'] / 365)
data['time_sin'] = np.sin(2 * np.pi * data['time'] / 1440)
data['time_cos'] = np.cos(2 * np.pi * data['time'] / 1440)

# Drop 'day' and 'time' columns
data = data.drop(['day', 'time'], axis=1)
data = data.rename(columns={'delivery_start': 'ds', target: 'y'})
df = data[['ds', 'y']]
df_train, df_test = train_test_split(df, test_size=0.1, random_state=42)

df.head()


,ds,y
0,2020-08-01 01:45:00,-285675
1,2020-08-01 02:00:00,-371020
2,2020-08-01 02:15:00,-423057
3,2020-08-01 02:30:00,-127568
4,2020-08-01 02:45:00,-130131


In [14]:
model = NeuralProphet(
    #n_forecasts=1,
    #n_lags=10,
    #yearly_seasonality=True,
    #weekly_seasonality=True,
    #daily_seasonality=True,
    #batch_size=64,
    #epochs=100,
    #learning_rate=1.0,
)
metrics = model.fit(df_train, freq="15min", epochs=20)

WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
WARNING - (py.warnings._showwarnmsg) - c:\Users\morit\miniconda3\envs\symreg\Lib\site-packages\neuralprophet\df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

INFO - (NP.df_utils._infer_frequency) - Major frequency -262530min corresponds to 0.007% of the data.
WARNING - (NP.df_utils._infer_frequency) - Dataframe has multiple frequencies. It will be resampled according to given freq 15min. Ignore                     message if actual frequency is any of the following:  SM, BM, CBM, SMS, BMS, CBMS, BQ, BQS, BA,                         or, BAS.
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.config.set_auto_batch_epoch)

Epoch 20: 100%|██████████| 20/20 [00:02<00:00,  6.93it/s, loss=0.0286, v_num=1, MAE=3.58e+5, RMSE=4.64e+5, Loss=0.0277, RegLoss=0.000]   


In [ ]:
from sklearn.metrics import mean_squared_error
num_rows = df_test.shape[0]
df_future = model.make_future_dataframe(df_train, periods=num_rows, n_historic_predictions=len(df_train))
forecast = model.predict(df_train)
model.plot(forecast)

